In [1]:
import influxdb_client, os, time,csv
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd

token = os.getenv('INFLUX_TOKEN')
org = os.getenv('ORG')
url = "http://localhost:8086"
bucket = os.getenv("BUCKET")

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)


In [2]:
path = r"C:\Users\rajpo\PycharmProjects\Tecnokrat\Time-series-models\Timeseries_dashboard\Electric_Production.csv"
name = path.split("\\")[-1].split(".csv")[-2].strip()

In [3]:
def write_data_to_influxdb(csv_file,bucket,measurement,org):
    check = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Get header row
        for row in reader:
            data = [
                {
                    "measurement": measurement,
                    "tags": {},
                    "fields": {headers[i]: row[i] for i in range(len(headers))},
                }
            ]

            check.append(data)
            # Write data to InfluxDB
            write_api = client.write_api(write_options=SYNCHRONOUS)
            write_api.write(bucket=bucket, org=org, record=data)

    return check

In [4]:
csv_file = path
bucket = "Time_series"
measurement = "s10"
org = "Teknocrat_S"

In [5]:
res1 = write_data_to_influxdb(csv_file,bucket,measurement,org)

In [6]:
res1

[[{'measurement': 's10',
   'tags': {},
   'fields': {'DATE': '1/1/1985', 'IPG2211A2N': '72.5052'}}],
 [{'measurement': 's10',
   'tags': {},
   'fields': {'DATE': '2/1/1985', 'IPG2211A2N': '70.672'}}],
 [{'measurement': 's10',
   'tags': {},
   'fields': {'DATE': '3/1/1985', 'IPG2211A2N': '62.4502'}}],
 [{'measurement': 's10',
   'tags': {},
   'fields': {'DATE': '4/1/1985', 'IPG2211A2N': '57.4714'}}],
 [{'measurement': 's10',
   'tags': {},
   'fields': {'DATE': '5/1/1985', 'IPG2211A2N': '55.3151'}}],
 [{'measurement': 's10',
   'tags': {},
   'fields': {'DATE': '6/1/1985', 'IPG2211A2N': '58.0904'}}],
 [{'measurement': 's10',
   'tags': {},
   'fields': {'DATE': '7/1/1985', 'IPG2211A2N': '62.6202'}}],
 [{'measurement': 's10',
   'tags': {},
   'fields': {'DATE': '8/1/1985', 'IPG2211A2N': '63.2485'}}],
 [{'measurement': 's10',
   'tags': {},
   'fields': {'DATE': '9/1/1985', 'IPG2211A2N': '60.5846'}}],
 [{'measurement': 's10',
   'tags': {},
   'fields': {'DATE': '10/1/1985', 'IPG2211

In [7]:
len(res1)

397

In [8]:
query_api  = client.query_api() # "select * from trial1"

In [15]:
flux_query = f'''
from(bucket: "{bucket}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "{measurement}")
'''


In [16]:
result = query_api.query(flux_query)

In [108]:
data = []
for table in result:
    for record in table.records:
        data.append(record.values)

if data:
    df_from_query = pd.DataFrame(data)
    #df.to_csv('trial1_data.csv', index=False)
else:
    print("No data returned from the query.")

In [109]:
df_from_query = df_from_query[["_value","_field"]]
df_from_query

,_value,_field
0,1/1/1985,DATE
1,2/1/1985,DATE
2,3/1/1985,DATE
3,4/1/1985,DATE
4,5/1/1985,DATE
...,...,...
789,98.6154,IPG2211A2N
790,93.6137,IPG2211A2N
791,97.3359,IPG2211A2N
792,114.7212,IPG2211A2N


In [125]:
all_df = []
for values in list(pd.unique(df_from_query._field)):
    print(values)
    group = df_from_query.groupby(by="_field").get_group(values)
    date_df = df_from_query[df_from_query['_field'] == values]

    date_df = date_df.rename(columns={'_value': values}).drop('_field', axis=1)
    # Reset index
    date_df = date_df.reset_index(drop=True)
    all_df.append(date_df)


DATE
IPG2211A2N


In [126]:
all_df[1]

,IPG2211A2N
0,72.5052
1,70.672
2,62.4502
3,57.4714
4,55.3151
...,...
392,98.6154
393,93.6137
394,97.3359
395,114.7212


In [127]:
combined_df = pd.concat(all_df,axis=1)

In [135]:
combined_df["IPG2211A2N"] = combined_df["IPG2211A2N"].astype("float")

In [136]:
combined_df

,DATE,IPG2211A2N
0,1/1/1985,72.5052
1,2/1/1985,70.6720
2,3/1/1985,62.4502
3,4/1/1985,57.4714
4,5/1/1985,55.3151
...,...,...
392,9/1/2017,98.6154
393,10/1/2017,93.6137
394,11/1/2017,97.3359
395,12/1/2017,114.7212


In [137]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DATE        397 non-null    object 
 1   IPG2211A2N  397 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.3+ KB


In [138]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DATE        397 non-null    object 
 1   IPG2211A2N  397 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.3+ KB


In [139]:
if df_orig.sort_index(axis=1).equals(combined_df.sort_index(axis=1)):
    print("DataFrames are the same")
else:
    print("DataFrames are different")

DataFrames are the same


In [140]:
if df_orig.equals(combined_df):
    print("DataFrames are the same")
else:
    print("DataFrames are different")

DataFrames are the same
